In [1]:
import deeplabcut

import yaml

from paramiko import SSHClient
import paramiko
from scp import SCPClient
import getpass

import os

In [2]:
# move folder from Shrek to local computer to label frames on local computer. Set shrek project folder and dest_folder
# Also may need to update ssh key/id

shrek_project_folder = '/home/jts3256/DLC_videos/Handle_tracking-Joe-2020-12-08'
dest_folder = r'D:\Lab\Data\DLC_videos\handle_labeling'

image_folder = shrek_project_folder + '/labeled-data'
config_file = shrek_project_folder + '/config.yaml'

if(not os.path.isdir(dest_folder)):
    os.mkdir(dest_folder)

ssh = SSHClient()
ssh.load_host_keys(filename = r'C:\Users\Joseph Sombeck\shrek_publickey (2).ppk')

pkey_filename = r'C:\Users\Joseph Sombeck\shrek_sshkey'
#pkey = paramiko.RSAKey.from_private_key_file(pkey_filename)
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())



ssh.connect(hostname = "165.124.111.57", username= "jts3256", key_filename=pkey_filename)
with SCPClient(ssh.get_transport(), sanitize=lambda x: x) as scp:
    scp.get(image_folder,recursive = True, local_path=dest_folder)
    scp.get(config_file,recursive = False, local_path=dest_folder)
    
local_config_path = dest_folder+ r'\config.yaml'

# update project path in config path
with open(local_config_path) as file:
    data = yaml.load(file, Loader = yaml.FullLoader)

    data['project_path'] = dest_folder # default = 20

with open(local_config_path,'w') as file:
    yaml.dump(data,file)


In [ ]:
"""
label frames
"""
#local_config_path = r'D:\Lab\Data\DLC_videos\Crackle-Qiwei-2020-12-03\config.yaml'
deeplabcut.label_frames(local_config_path,imtypes=['*png'])

In [4]:
#deeplabcut.check_labels(local_config_path)

# move files back onto shrek
ssh = SSHClient()
ssh.load_host_keys(filename = r'C:\Users\Joseph Sombeck\shrek_publickey (2).ppk')

pkey_filename = r'C:\Users\Joseph Sombeck\shrek_sshkey'
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(hostname = "165.124.111.57", username= "jts3256", key_filename=pkey_filename)
scp  = SCPClient(ssh.get_transport())

scp.put(files = dest_folder + '\labeled-data',recursive = True, 
        remote_path =shrek_project_folder)
